# Setup

In [43]:
#!pip install transformers datasets accelerate

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Pipeline method lets us use any huggingface model with a few lines of code.

[Tutorial](https://huggingface.co/docs/transformers/v4.28.1/en/pipeline_tutorial)

[Details on API usage](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/pipelines#transformers.pipeline)

[Details on NLP Task Pipelines](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/pipelines#natural-language-processing)[link text](https://)


In [44]:
import torch

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from datasets import load_dataset

# Default pipeline

The pipeline() is the easiest and fastest way to use a pretrained model for inference. You can use the pipeline() out-of-the-box for many tasks across different modalities. [Details](https://huggingface.co/docs/transformers/v4.28.1/en/quicktour)


## Sentiment Classification

In [7]:
# The pipeline() downloads and caches a default pretrained model and tokenizer for sentiment analysis. Now you can use the classifier on your target text:
classifier = pipeline(task="sentiment-analysis")
classifier("This movie is really good.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998592138290405}]

## Summarization

In [9]:
summarizer = pipeline(task="summarization", device_map="auto")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
text = """
The Federal Reserve on Friday released its assessment of what led to Silicon Valley Bank's 
collapse, saying the lender's failure was due to a "textbook case of mismanagement" 
while taking some responsibility for insufficient supervision of the institution.
The report details the bank’s rapid growth, the challenges Fed supervisors faced in 
identifying SVB's vulnerabilities and their reluctance to force the bank to fix them. 
The review was led by Fed Vice Chair for Supervision Michael S. Barr, who wrote in a 
letter summarizing the report that SVB "failed because of a textbook case of 
mismanagement by the bank. Its senior leadership failed to manage basic 
interest rate and liquidity risk. Its board of directors failed to oversee senior 
leadership and hold them accountable."
The report said SVB's leadership did not fully appreciate the bank's vulnerabilities, 
pointing to foundational and widespread managerial weaknesses, its highly concentrated 
business model catering overwhelmingly to the venture capital community, and its reliance 
on uninsured deposits which left the bank "acutely exposed to rising interest rates" 
amid a slowdown in the tech sector.
"""

In [11]:
summarizer(text)

[{'summary_text': " Federal Reserve releases its assessment of what led to Silicon Valley Bank's collapse . Report details the bank’s rapid growth, the challenges Fed supervisors faced in identifying SVB's vulnerabilities and their reluctance to force the bank to fix them . Review was led by Fed Vice Chair for Supervision Michael S. Barr, who wrote in a letter summarizing the report ."}]

## Text Generation

In [12]:
txt_gen = pipeline(task="text-generation", device_map="auto")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
txt_gen("The capital of India is")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'The capital of India is based on the name of India, or it has been referred to as "India" or "India by its neighbors" for decades. In the South the most famous name the capital holds is "Dalits", but for'}]

# Pipeline with specific models


## Sentiment Classification

In [15]:
# find models for tasks at: https://huggingface.co/models
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
classifier = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer, device_map="auto")

## Example usage over entire dataset

In [34]:
# find datasets for tasks at: https://huggingface.co/datasets
sst2_dataset = load_dataset("sst2", split="test")

In [35]:
# here we explicitly extract the sentences
text, labels = dataset["sentence"], dataset["label"]
sst2_results = classifier(text)

In [37]:
# KeyDataset is a util that will just output the item we're interested in.
from transformers.pipelines.pt_utils import KeyDataset

for out in classifier(KeyDataset(sst2_dataset, "sentence")):
    print(out)
    break

{'label': 'NEGATIVE', 'score': 0.9996720552444458}


## Question-Answering

In [38]:
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [39]:
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example     of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

In [40]:
result = question_answerer(question="What is a good example of a question answering dataset?",\
                           context=context)

In [41]:
result

{'score': 0.5152307152748108,
 'start': 151,
 'end': 164,
 'answer': 'SQuAD dataset'}

# Using pipeline with large language models

You can easily run pipeline on large models using 🤗 accelerate! First make sure you have installed accelerate with pip install accelerate.

First load your model using device_map="auto"! We will use facebook/opt-1.3b for our example.

In [45]:
# pip install accelerate
pipe = pipeline(model="facebook/opt-1.3b", torch_dtype=torch.bfloat16, device_map="auto")

In [46]:
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
output

[{'generated_text': "This is a cool example! It's got really neat animation to show its progress convoluting along with"}]